In [26]:
%run default-imports.ipynb

In [27]:
%run run-experiment-cv.ipynb

In [28]:
filenames = {'MIMIC' : "~/cohorts/hs_mimic.csv", 'SINAI' : "~/cohorts/hs_sinai_preprocessed.csv", 'DHZB' : "~/cohorts/hs_dhzb.csv"}
save_path = './experiments/dhzb_without_pipelines.d'

In [30]:
''' define options to run experiments on '''
options = {
    'target': ["AKI"],
    'cohort': ['DHZB'],
    'test_size': [0.2],
    'imputation_method':[imputers.DEFAULT],    
    'algorithm': [
        algorithms.LR,
        algorithms.DT,
        algorithms.ADABOOST,
        algorithms.ENLR,
        algorithms.GNBAYES,
        algorithms.GBDT,
        algorithms.RF
    ],
    'sampling_method': [samplers.SMOTE],
    'sampling_strategy': [0.25],
    'scale_method': [None],
    'calibration_method':['isotonic','sigmoid'],
    'save_pipeline':[False],
    'optimize_mode': [False],
    'n_splits':[10]
}

experiments = unpickle(save_path) or {}
n_experiments = 0

with Timer() as t:
    
    ''' iterate over different options '''
    for combination in product(*options.values()):

        ''' initialize parameters '''
        params = dict(zip(options.keys(), combination))
        print(f"Running experiment with following parameters: {params}")
        exp_id = str(uuid.uuid1())
        experiment = defaultdict(lambda: {})        

        ''' load the data '''
        data = Load().execute(filename=filenames[params['cohort']])
        
        ''' split the data '''
        train, test = Split().execute(data,test_size=params['test_size'])        
        
        ''' train model on subset of data '''
        experiment = run_experiment(params, train, test)
                
        ''' after training, our data for subsequent steps becomes train for computing CV statistics '''
        data = train.copy()
        
        features, labels = data.drop(params['target'], axis=1), data[params['target']]                
        
        ''' initialize metric storage '''
        metrics = ['precision', 'recall', 'f1-score', 'auc', 'dor']
        cv_experiments = []
        cv_metrics = defaultdict(lambda: {})
        cv_performance = defaultdict(lambda: {})
        
        k_params = params
        del k_params['calibration_method'] #no need to calibrate within the k-folds
        
        ''' for each of the k-folds '''
        skf = StratifiedKFold(n_splits=params['n_splits'], random_state=None, shuffle=False)        
        for train_index, test_index in skf.split(features, labels):
            train_data = data.iloc[train_index]
            test_data = data.iloc[test_index]
            cv_experiments.append(run_experiment(k_params, train_data, test_data))

        ''' summarize results of crossvalidation '''        
        for metric in metrics:                       
            measurements = [exp['performance']['discrimination'][metric] for exp in cv_experiments]
            cv_performance[metric]['mean'] = np.mean(measurements)
            cv_performance[metric]['std'] = np.std(measurements)
            cv_performance[metric]['ci'] = np.std(measurements) * 2 #95% CI
        
        ''' save everything '''
        experiment['parameters'] = params
        experiment['performance']['cv'] = cv_performance
        experiment['exp_id'] = exp_id
        
        experiments[exp_id] = experiment
        
        n_experiments += 1

print(f'Running {n_experiments} experiments took {t.interval:.03f} sec.')

''' store everything '''
if pickle(experiments, save_path): print('Successfully saved.')

Could not unpickle object. [Errno 2] No such file or directory: './experiments/sinai_model_results.d'
Running experiment with following parameters: {'target': 'AKI', 'cohort': 'SINAI', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'LogisticRegression', 'sampling_method': 'SMOTE', 'sampling_strategy': 0.25, 'scale_method': None, 'calibration_method': 'isotonic', 'save_pipeline': False, 'optimize_mode': False, 'n_splits': 10}
Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=19721
Class:1.0/N=918
Label distribution after sampling: 
Class:0.0/N=19721
Class:1.0/N=4930
*** Training of model 'LogisticRegression' started.


lbfgs failed to converge. Increase the number of iterations.


*** Training of classifier ready. Time elapsed: 433.018ms

Performing calibration for LogisticRegression
Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17748
Class:1.0/N=826
Label distribution after sampling: 
Class:0.0/N=17748
Class:1.0/N=4437
*** Training of model 'LogisticRegression' started.


lbfgs failed to converge. Increase the number of iterations.


*** Training of classifier ready. Time elapsed: 210.288ms

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17749
Class:1.0/N=826
Label distribution after sampling: 
Class:0.0/N=17749
Class:1.0/N=4437
*** Training of model 'LogisticRegression' started.


...

divide by zero encountered in double_scalars


Label distribution after sampling: 
Class:0.0/N=17716
Class:1.0/N=4429
*** Training of model 'GaussianNB' started.
*** Training of classifier ready. Time elapsed: 28.614ms

Running experiment with following parameters: {'target': 'AKI', 'cohort': 'SINAI', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'GradientBoostingDecisionTree', 'sampling_method': 'SMOTE', 'sampling_strategy': 0.25, 'scale_method': None, 'calibration_method': 'isotonic', 'save_pipeline': False, 'optimize_mode': False, 'n_splits': 10}


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
invalid value encountered in long_scalars
invalid value encountered in double_scalars
invalid value encountered in sqrt
invalid value encountered in double_scalars


Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=19700
Class:1.0/N=939
Label distribution after sampling: 
Class:0.0/N=19700
Class:1.0/N=4925
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 97.989ms

Performing calibration for GradientBoostingDecisionTree
Performing sampling with SMOTE...
Prior label distribution: 
Class:1.0/N=845
Class:0.0/N=17730
Label distribution after sampling: 
Class:1.0/N=4432
Class:0.0/N=17730
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 532.245ms

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17730
Class:1.0/N=845
Label distribution after sampling: 
Class:0.0/N=17730
Class:1.0/N=4432
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 610.719ms

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17730
Class:1.0/N

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17757
Class:1.0/N=818
Label distribution after sampling: 
Class:0.0/N=17757
Class:1.0/N=4439
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 454.059ms

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17757
Class:1.0/N=818
Label distribution after sampling: 
Class:0.0/N=17757
Class:1.0/N=4439
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 798.822ms

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17757
Class:1.0/N=818
Label distribution after sampling: 
Class:0.0/N=17757
Class:1.0/N=4439
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 790.087ms

Performing sampling with SMOTE...
Prior label distribution: 
Class:0.0/N=17757
Class:1.0/N=818
Label distribution after sampling: 
Class:0.0/N=17757
Class:1.